In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Télécharger Apache Spark (vérifier la dernière version si besoin)
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

# Installer les packages Python
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install -q pymongo matplotlib seaborn

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
3 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open: No suc

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("ColabSpark") \
.config("spark.driver.memory", "2g") \
.getOrCreate()
print(" Spark est configuré avec succès !")

 Spark est configuré avec succès !


In [4]:
print(spark.version)

4.0.1


In [8]:
data = [(1, "Alice", 23), (2, "Bob", 30), (3, "Charlie", 29)]
columns = ["id", "nom", "age"]

df = spark.createDataFrame(data, columns)

df.show()


+---+-------+---+
| id|    nom|age|
+---+-------+---+
|  1|  Alice| 23|
|  2|    Bob| 30|
|  3|Charlie| 29|
+---+-------+---+



In [9]:
df.printSchema()                 # Affiche la structure
df.select("nom", "age").show()   # Sélectionne certaines colonnes
df.filter(df.age > 25).show()    # Filtre les lignes où age > 25

root
 |-- id: long (nullable = true)
 |-- nom: string (nullable = true)
 |-- age: long (nullable = true)

+-------+---+
|    nom|age|
+-------+---+
|  Alice| 23|
|    Bob| 30|
|Charlie| 29|
+-------+---+

+---+-------+---+
| id|    nom|age|
+---+-------+---+
|  2|    Bob| 30|
|  3|Charlie| 29|
+---+-------+---+



In [12]:
df.printSchema() # Structure du DataFrame
df.select("nom", "age").show() # Sélection de colonnes
df.filter(df.age > 25).show() # Filtrage des données

root
 |-- id: long (nullable = true)
 |-- nom: string (nullable = true)
 |-- age: long (nullable = true)

+-------+---+
|    nom|age|
+-------+---+
|  Alice| 23|
|    Bob| 30|
|Charlie| 29|
+-------+---+

+---+-------+---+
| id|    nom|age|
+---+-------+---+
|  2|    Bob| 30|
|  3|Charlie| 29|
+---+-------+---+



In [14]:
df = spark.read.csv("/content/transaction_data.csv", header=True, inferSchema=True)
df.show(5)

+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
|Transaction ID|Sender Account ID|Receiver Account ID|Transaction Amount|Transaction Type|          Timestamp|Transaction Status|Fraud Flag|Geolocation (Latitude/Longitude)|Device Used|Network Slice ID|Latency (ms)|Slice Bandwidth (Mbps)|PIN Code|
+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
| TXN9520068950|         ACC14994|           ACC16656|             495.9|         Deposit|2025-01-17 10:14:00|            Failed|      true|            34.0522 N, -74.006 W|    Desktop|          Slice3|          10|                   179|    3075|
| TXN941

In [15]:
df.printSchema()

root
 |-- Transaction ID: string (nullable = true)
 |-- Sender Account ID: string (nullable = true)
 |-- Receiver Account ID: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Transaction Type: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- Transaction Status: string (nullable = true)
 |-- Fraud Flag: boolean (nullable = true)
 |-- Geolocation (Latitude/Longitude): string (nullable = true)
 |-- Device Used: string (nullable = true)
 |-- Network Slice ID: string (nullable = true)
 |-- Latency (ms): integer (nullable = true)
 |-- Slice Bandwidth (Mbps): integer (nullable = true)
 |-- PIN Code: integer (nullable = true)



In [16]:
df.filter(df["Transaction Amount"] > 1000).show()

+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
|Transaction ID|Sender Account ID|Receiver Account ID|Transaction Amount|Transaction Type|          Timestamp|Transaction Status|Fraud Flag|Geolocation (Latitude/Longitude)|Device Used|Network Slice ID|Latency (ms)|Slice Bandwidth (Mbps)|PIN Code|
+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
| TXN2214150284|         ACC48650|           ACC76457|           1129.88|        Transfer|2025-01-17 10:56:00|           Success|      true|            34.0522 N, -74.006 W|     Mobile|          Slice3|          10|                   127|    6374|
| TXN310

In [17]:
df.groupBy("Transaction Type").sum("Transaction Amount").show()

+----------------+-----------------------+
|Transaction Type|sum(Transaction Amount)|
+----------------+-----------------------+
|         Deposit|     252042.61999999988|
|        Transfer|      291776.5500000002|
|      Withdrawal|     227346.12000000002|
+----------------+-----------------------+



In [18]:
df.orderBy(df["Transaction Amount"].desc()).show(5)

+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
|Transaction ID|Sender Account ID|Receiver Account ID|Transaction Amount|Transaction Type|          Timestamp|Transaction Status|Fraud Flag|Geolocation (Latitude/Longitude)|Device Used|Network Slice ID|Latency (ms)|Slice Bandwidth (Mbps)|PIN Code|
+--------------+-----------------+-------------------+------------------+----------------+-------------------+------------------+----------+--------------------------------+-----------+----------------+------------+----------------------+--------+
| TXN1696527060|         ACC44804|           ACC26269|           1497.76|        Transfer|2025-01-17 10:54:00|           Success|      true|            55.7558 N, -118.2...|    Desktop|          Slice2|          10|                   113|    5668|
| TXN874